# AN2DL - Colab Notebook

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/

Mounted at /gdrive
/gdrive/My Drive


## ⚙️ Import Libraries

### Dependencies for training

In [ ]:
!pip install keras_cv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 51.4 MB/s eta 0:00:00


In [ ]:
import numpy as np

import tensorflow as tf

import keras as tfk

from keras.layers import LeakyReLU, Dense, Flatten, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import AdamW
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from sklearn.model_selection import train_test_split

import keras.layers as tfkl

import keras_cv as kcv
from keras_cv.layers import RandomApply

import matplotlib.pyplot as plt

from keras.applications import ConvNeXtBase


np.random.seed(42)
tf.random.set_seed(42)

## Data loading and preprocessing

### Load data

In [ ]:
data = np.load('data_unique.npz', allow_pickle=True)
X = data['images']
y = data['labels']

In [ ]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X,
                                                                      y,
                                                                      test_size=0.2,
                                                                      random_state=42,
                                                                      stratify=y)

### Augmentation pipeline

In [ ]:
random_layers = [
    RandomApply(kcv.layers.RandAugment((0, 255)), rate=0.9),
    RandomApply(kcv.layers.JitteredResize((96, 96), (0.9, 1)), rate=0.7),
    RandomApply(kcv.layers.RandomTranslation(0.4, 0.4), rate=0.6),
    RandomApply(kcv.layers.RandomRotation(1), rate=0.6),
    RandomApply(kcv.layers.RandomSaturation(0.7), rate=0.3),
    RandomApply(kcv.layers.RandomContrast((0, 255), 0.7), rate=0.3),
    RandomApply(kcv.layers.RandomCutout(0.6, 0.6), rate=0.5)
]
augmenter = tfk.Sequential(random_layers)

In [ ]:
# Build val dataset for transfer learning (batch of 32)
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32).prefetch(tf.data.AUTOTUNE)
train_ds = train_ds.map(
    lambda x, y: (augmenter((x)), y),
    num_parallel_calls=tf.data.AUTOTUNE
)

# Build val dataset for transfer learning (batch of 32)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.map(
    lambda x, y: ((x), y),
    num_parallel_calls=tf.data.AUTOTUNE
)

### Visualize augmented images

In [ ]:
def display_augmented_images(dataset, n_images=10):
    plt.figure(figsize=(30, 10))
    for i, (images, labels) in enumerate(dataset.take(1)):
        # Apply augmentation to the first n_images
        augmented_images = augmenter(images[:n_images])
        for j in range(n_images):
            ax = plt.subplot(1, n_images, j + 1)
            plt.imshow(tf.cast(augmented_images[j], tf.uint8))
            plt.axis("off")
        break  # Take only the first batch

display_augmented_images(train_ds)
plt.show()

## 🛠️ Train and Save the Model

### Transfer learning

In [ ]:
def create_model():
    # Load ConvNeXt model
    base_model = ConvNeXtBase(weights='imagenet', include_top=False, input_shape=(96, 96, 3))

    # Deactivate all layers for transfer learning
    for layer in base_model.layers:
        layer.trainable = False

    # Classifier
    x = base_model.output
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(8, activation='softmax')(x)  # 8 classes

    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer=AdamW(learning_rate=1e-4,
                                    weight_decay=1e-6,
                                    epsilon=1e-8,
                                    beta_1=0.9,
                                    beta_2=0.999,
                                  ), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = create_model()

In [ ]:
# Early stopping to stop training and save best model, lr scheduler for best performance
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-7)

# Training
history = model.fit(
    train_ds,
    epochs=30,
    batch_size=32,
    validation_data=(val_ds),
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
model.save('tf.keras')

### Fine Tuning pt. 1

In [ ]:
model = tf.keras.models.load_model('tf.keras')

In [ ]:
# Build train dataset for first round of fine tuning (batch of 128)
train_ds_ft = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(128).prefetch(tf.data.AUTOTUNE)
train_ds_ft = train_ds_ft.map(
    lambda x, y: (augmenter((x)), y),
    num_parallel_calls=tf.data.AUTOTUNE
)

# Build val dataset for first round of fine tuning (batch of 128)
val_ds_ft = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(128).prefetch(tf.data.AUTOTUNE)
val_ds_ft = val_ds_ft.map(
    lambda x, y: ((x), y),
    num_parallel_calls=tf.data.AUTOTUNE
)

In [ ]:
# Activate last 10 layers of the network
for layer in model.layers[-10:]:
        layer.trainable = True

model.compile(optimizer=AdamW(learning_rate=1e-4,
                                    weight_decay=1e-5,
                                    epsilon=1e-8,
                                    beta_1=0.9,
                                    beta_2=0.999,
                                  ), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=1, min_lr=1e-8)

history = model.fit(
    train_ds_ft,
    epochs=40,
    batch_size=128,
    validation_data=(val_ds_ft),
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:
model.save('ft1.keras')

### Fine Tuning pt. 2

In [ ]:
model = tf.keras.models.load_model('ft1.keras')

In [ ]:
train_ds_ft = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(128).prefetch(tf.data.AUTOTUNE)
train_ds_ft = train_ds_ft.map(
    lambda x, y: (augmenter((x)), y),
    num_parallel_calls=tf.data.AUTOTUNE
)

val_ds_ft = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(128).prefetch(tf.data.AUTOTUNE)
val_ds_ft = val_ds_ft.map(
    lambda x, y: ((x), y),
    num_parallel_calls=tf.data.AUTOTUNE
)

In [ ]:
# Enable entire network
for layer in model.layers:
        layer.trainable = True

model.compile(optimizer=AdamW(learning_rate=1e-4,
                                    weight_decay=1e-5,
                                    epsilon=1e-8,
                                    beta_1=0.9,
                                    beta_2=0.999,
                                  ), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=20, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=1, min_lr=1e-8)

history = model.fit(
    train_ds_ft,
    epochs=40,
    batch_size=128,
    validation_data=(val_ds_ft),
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/40
75/75 ━━━━━━━━━━━━━━━━━━━━ 296s 3s/step - accuracy: 0.7943 - loss: 0.5818 - val_accuracy: 0.9737 - val_loss: 0.0799 - learning_rate: 1.0000e-04
Epoch 2/40
75/75 ━━━━━━━━━━━━━━━━━━━━ 183s 2s/step - accuracy: 0.8716 - loss: 0.3700 - val_accuracy: 0.9799 - val_loss: 0.0618 - learning_rate: 1.0000e-04
Epoch 3/40
75/75 ━━━━━━━━━━━━━━━━━━━━ 162s 2s/step - accuracy: 0.8880 - loss: 0.3268 - val_accuracy: 0.9795 - val_loss: 0.0542 - learning_rate: 1.0000e-04
Epoch 4/40
75/75 ━━━━━━━━━━━━━━━━━━━━ 164s 2s/step - accuracy: 0.9052 - loss: 0.2756 - val_accuracy: 0.9854 - val_loss: 0.0475 - learning_rate: 1.0000e-04
Epoch 5/40
75/75 ━━━━━━━━━━━━━━━━━━━━ 161s 2s/step - accuracy: 0.9167 - loss: 0.2396 - val_accuracy: 0.9854 - val_loss: 0.0402 - learning_rate: 1.0000e-04
Epoch 6/40
75/75 ━━━━━━━━━━━━━━━━━━━━ 202s 2s/step - accuracy: 0.9185 - loss: 0.2332 - val_accuracy: 0.9854 - val_loss: 0.0454 - learning_rate: 1.0000e-04
Epoch 7/40
75/75 ━━━━━━━━━━━━━━━━━━━━ 161s 2s/step - accuracy: 0.9212 

KeyboardInterrupt: 

In [ ]:
model.save('weights.keras')